In [ ]:
from IPython import display
!pip install transformers
!pip install sacrebleu
!pip install sacremoses
!pip install datasets
!pip install wandb
!pip install sentencepiece
!pip install nltk>=3.4.5
!pip install numpy requests nlpaug
!wget https://raw.githubusercontent.com/SunbirdAI/nmt_training/main/salt_v2/salt.py
!wget https://raw.githubusercontent.com/SunbirdAI/nmt_training/main/nmt_clean/augmentations.py
display.clear_output()

In [ ]:
from augmentations import Augmentations
import datasets
from IPython import display
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
from nlpaug.util.file.download import DownloadUtil
import numpy as np
import os
import pandas as pd
import random
import sentencepiece
import sacrebleu
import sacremoses
import salt
import tqdm
import transformers
import torch
import wandb


In [ ]:
!mkdir /kaggle/temp

In [ ]:
#DownloadUtil.download_word2vec(dest_dir='/kaggle/temp') # Download word2vec model
#DownloadUtil.download_glove(model_name='glove.6B', dest_dir='/kaggle/temp') # Download GloVe model
DownloadUtil.download_fasttext(model_name='wiki-news-300d-1M', dest_dir='/kaggle/temp') # Download fasttext model


In [ ]:
#embeddings_augmentation = naw.WordEmbsAug(model_type='fasttext',
#                      model_path="/kaggle/temp/wiki-news-300d-1M.vec",top_k=10, aug_p=0.05, aug_min=0,aug_max=2)
swap_augmentation = naw.random.RandomWordAug("swap", aug_p=0.015, aug_min=0,aug_max=1)
delete_augmentation = naw.random.RandomWordAug("delete", aug_p=0.015, aug_min=0,aug_max=1)

# sometimes_augmenter = naf.sometimes.Sometimes([
#     embeddings_augmentation, 
#     swap_augmentation,
#     delete_augmentation
    
# ])

multiple_augs = Augmentations(
[
    #embeddings_augmentation,
    swap_augmentation,
    delete_augmentation
]
)

In [ ]:
augmented_text = multiple_augs([
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck",
    "Testing again is this shit too ducking slow fuck"
])

In [ ]:
augmented_text

In [ ]:
# aug_p (float) – Percentage of word will be augmented.
# aug_min (int) – Minimum number of word will be augmented.
# aug_max (int) – Maximum number of word will be augmented. If None is passed, number of augmentation is calculated via aup_p. If calculated result from aug_p is smaller than aug_max, will use calculated result from aug_p. Otherwise, using aug_max.

In [ ]:
torch.cuda.is_available()

In [ ]:
# Parameters for mul-en models
config = {
    'source_language': 'eng',
    'target_language': 'many',
    'metric_for_best_model': 'loss',
    'effective_train_batch_size': 5000,
    'max_input_length': 128,
    'validation_samples_per_language': 500,
    'eval_languages': ["ach", "lgg", "lug", "nyn", "teo", "luo"],
    'eval_pretrained_model': True,
    'learning_rate': 5e-4,
    'num_train_epochs': 10,
    'label_smoothing_factor': 0.1,
    'mt560_relative_sample_rate' : 0.2,
    'flores200_training_data': True,
    'mt560_training_data': True,
    'monolingual_training_data': False,
    'back_translation_training_data': True,
    'google_back_translation_data': True,
    'named_entities_training_data': False,
    'lafand_training_data': True,
    'tag_subsets': True,
    'aug_subsets': True,
    'early_stopping_patience': 4,
    'eval_steps_interval': 50,
    'data_dir': 'salt-translation-plus-external-datasets-15-3-23',
}

config['language_pair'] = (f'{config["source_language"]}-'
                           f'{config["target_language"]}')
config['wandb_project'] = f'sunbird-translate-{config["language_pair"]}'
config['model_checkpoint'] = f'Helsinki-NLP/opus-mt-en-mul'
#config['model_checkpoint'] = f'/kaggle/input/nmt-marianmt-w-mafand/best/checkpoint-600'
#config['model_checkpoint'] = (
#    '/content/gdrive/MyDrive/Translation/saved_models/'
#    'marianmt-many-eng/checkpoint-1400')


# Find the biggest batch size that fits in GPU memory
APPROX_MODEL_MEMORY_SIZE_MB = 310
if torch.cuda.is_available():
  gpu_info = !nvidia-smi
  gpu_memory_mb = int(gpu_info[9].split()[10][:-3])
  per_device_max_batch_size = int(gpu_memory_mb / APPROX_MODEL_MEMORY_SIZE_MB)
  B = config['effective_train_batch_size'] 
  factors = np.array([x for x in range(1, B) if B % x == 0])
  config['train_batch_size'] = int(max(
      factors[factors < per_device_max_batch_size]))
  config['eval_batch_size'] = config['train_batch_size']
else:
  config['train_batch_size'] = 1
  config['eval_batch_size'] = 1
config['gradient_accumulation_steps'] = int(
    config['effective_train_batch_size'] / config['train_batch_size'])

# Trainer settings
config['train_settings'] = transformers.Seq2SeqTrainingArguments(
    output_dir= f'/kaggle/working/best',
    evaluation_strategy = 'steps',
    eval_steps = config['eval_steps_interval'],
    save_steps = config['eval_steps_interval'],
    gradient_accumulation_steps = config['gradient_accumulation_steps'],
    learning_rate = config['learning_rate'],
    per_device_train_batch_size = config['train_batch_size'],
    per_device_eval_batch_size = config['eval_batch_size'],
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = config['num_train_epochs'],
    predict_with_generate = True,
    fp16 = torch.cuda.is_available(),
    logging_dir = f'/kaggle/working/best',
    report_to = 'wandb',
    run_name = f'{config["source_language"]}-{config["target_language"]}',
    load_best_model_at_end=True,
    metric_for_best_model = config['metric_for_best_model'],
    label_smoothing_factor = config['label_smoothing_factor']
)

In [ ]:
config['training_subset_ids'] = ['salt-train', 'ai4d']

if config['lafand_training_data']:
    config['training_subset_ids'].extend(['lafand-en-lug-combined', 'lafand-en-luo-combined'])

if config['flores200_training_data']:
    config['training_subset_ids'] .append('flores200')

if config['back_translation_training_data']:
    config['training_subset_ids'].extend( ['bt_ach_en_14_3_23', 'bt_lug_en_14_3_23'])

if config['back_translation_training_data']:
    config['training_subset_ids'].extend(['backtranslated-from-eng-google', 'backtranslated-from-lug-google' ])

config['training_subset_ids'] = config['training_subset_ids']*3

if config['mt560_training_data']:
    config['training_subset_ids'].extend([
        'mt560_ach', 'mt560_lug', 'mt560_nyn','mt560_luo'])



In [ ]:
if config["tag_subsets"]:
    config['training_subset_tags'] = ["hq", "hq"]

    if config['lafand_training_data']:
        config['training_subset_tags'].extend([ "hq","hq" ])

    if config['flores200_training_data']:
        config['training_subset_tags'] .append("hq") #hq?

    if config['back_translation_training_data']:
        config['training_subset_tags'].extend( [ "bt", "bt" ])

    if config['back_translation_training_data']:
        config['training_subset_tags'].extend([ "ggl", "ggl" ])

    config['training_subset_tags'] = config['training_subset_tags']*3

    if config['mt560_training_data']:
        config['training_subset_tags'].extend([
            "ood", "ood", "ood", "ood" ])


In [ ]:
if config["aug_subsets"]:
    config['training_subset_augs'] = [True, True]

    if config['lafand_training_data']:
        config['training_subset_augs'].extend([ True,True ])

    if config['flores200_training_data']:
        config['training_subset_augs'] .append(True) #hq?

    if config['back_translation_training_data']:
        config['training_subset_augs'].extend( [ True, True ])

    if config['back_translation_training_data']:
        config['training_subset_augs'].extend([ True, True ])

    config['training_subset_augs'] = config['training_subset_augs']*3

    if config['mt560_training_data']:
        config['training_subset_augs'].extend([
            False, False, False, False ])

In [ ]:
assert len(config["training_subset_tags"]) == len(config["training_subset_ids"])

In [ ]:
if not os.path.exists('salt-translation-plus-external-datasets-15-3-23'):
    !wget https://sunbird-translate.s3.us-east-2.amazonaws.com/salt-translation-plus-external-datasets-15-3-23.zip
    !unzip salt-translation-plus-external-datasets-15-3-23.zip
    display.clear_output()

In [ ]:
from tqdm import tqdm
# training_subsets = [
#     salt.translation_dataset(
#         path=f'{config["data_dir"]}/{id}.jsonl',
#         source_language=config['source_language'],
#         target_language=config['target_language'],
#         allow_target_language_in_source=False,
#         prefix_target_language_in_source=True,
#         dataset_prefixes = dataset_tags,
#         languages_to_include=config['eval_languages'],
#         keep_unaugmented_src = False)
#     for id, dataset_tags in tqdm(zip(config['training_subset_ids'],config['training_subset_tags'] ) )
# ]

training_subsets = []

for id,aug_flag, dataset_tags in tqdm(zip(config['training_subset_ids'],config['training_subset_augs'],config['training_subset_tags'] )):
    for language in config["eval_languages"]:
        training_subset = salt.translation_dataset(
            path=f'{config["data_dir"]}/{id}.jsonl',
            source_language=config['source_language'],
            target_language=language,
            allow_target_language_in_source=False,
            prefix_target_language_in_source=False,
            dataset_prefixes = [f">>{language}_{dataset_tags}<<"],
            languages_to_include=language,
            keep_unaugmented_src = False)
        training_subsets.append(training_subset)
        if aug_flag:
            training_subset = salt.translation_dataset(
            path=f'{config["data_dir"]}/{id}.jsonl',
            source_language=config['source_language'],
            target_language=language,
            allow_target_language_in_source=False,
            prefix_target_language_in_source=False,
            source_augmenter = multiple_augs,
            dataset_prefixes = [f">>{language}_aug_{dataset_tags}<<"],
            languages_to_include=language,
            keep_unaugmented_src = False)
            training_subsets.append(training_subset)

In [ ]:
# sample_probabilities = np.array([len(s) for s in training_subsets])
# sample_probabilities = sample_probabilities * np.array(
#     [config['mt560_relative_sample_rate'] if ('mt560' in id) or ('google' in id)  else 1.0
#      for id in config['training_subset_ids']])
# sample_probabilities = sample_probabilities / np.sum(sample_probabilities)

# train_data_raw = datasets.interleave_datasets(
#     training_subsets, sample_probabilities)
train_data_raw = datasets.concatenate_datasets(training_subsets)
train_data_raw = train_data_raw.shuffle()
train_data_raw = train_data_raw.flatten_indices()

In [ ]:
validation_subsets = [
    salt.translation_dataset(
        path=f'{config["data_dir"]}/salt-dev.jsonl',
        source_language="eng",
        target_language=language,
        keep_unaugmented_src = False,
        allow_target_language_in_source=False,
        prefix_target_language_in_source=False,
        dataset_prefixes = [f">>{language}_hq<<"]
    )
    for language in config['eval_languages'][:-1]
]
validation_data_raw = datasets.concatenate_datasets(validation_subsets)

In [ ]:
def sentence_format(input):
    '''Ensure capital letter at the start and full stop at the end.'''
    try:
        input = input[0].capitalize() + input[1:]
        if input[-1] not in ['.', '!', '?']:
            input = input + '.'
    except:
        return ""
    return input

def preprocess(examples):
    normalizer = sacremoses.MosesPunctNormalizer()  
    inputs = []
    targets = []
    for input, target in zip(examples['source'], examples['target']):
        if not len(input):
          input = target
        inputs.append(sentence_format(normalizer.normalize(input)))
        targets.append(sentence_format(normalizer.normalize(target)))
    
    model_inputs = tokenizer(
        inputs, text_target=targets,
        max_length=config['max_input_length'], truncation=True)

    return model_inputs

def postprocess(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds, eval_languages, samples_per_language):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess(decoded_preds, decoded_labels)
    result = {}
    for i, lang in enumerate(eval_languages):
        
        result_subset = metric.compute(
            predictions=decoded_preds[
                i*samples_per_language:(i+1)*samples_per_language],
            references=decoded_labels[
                i*samples_per_language:(i+1)*samples_per_language])
        result[f"BLEU_{lang}"] = result_subset["score"]
        
    result["BLEU_mean"] = np.mean(
        [result[f"BLEU_{lang}"] for lang in eval_languages])
    
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(config['model_checkpoint'])
tokenizer = transformers.AutoTokenizer.from_pretrained(config['model_checkpoint'])
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer, model = model) 
metric = datasets.load_metric('sacrebleu')

In [ ]:
if config['target_language'] == 'many':
    replacements = {'nyn_ood': 'ewe',
                    'nyn_ggl': 'sna',
                    'nyn_hq': 'lin',
                    'nyn_bt': 'toi_Latn',
                    #'nyn_aug_ood': 'ceb',
                    'nyn_aug_ggl': 'oss',
                    'nyn_aug_hq':'run' ,
                    'nyn_aug_bt': 'mfe',
                    'teo_ood': 'ilo',
                    'teo_ggl': 'zlm_Latn',
                    'teo_hq': 'pes', 
                    'teo_bt': 'smo',
                    #'teo_aug_ood': 'hil',
                    'teo_aug_ggl': 'niu',
                    'teo_aug_hq': 'sag', 
                    'teo_aug_bt': 'fij',
                    'lgg_ood': 'cmn_Hans',
                    'lgg_ggl': 'nya',
                    'lgg_bt': 'tso',
                    'lgg_hq': 'war',
                    #'lgg_aug_ood': 'gil',
                    'lgg_aug_ggl': 'hau_Latn',
                    'lgg_aug_bt': 'umb',
                    'lgg_aug_hq': 'glv',
                    'ach_ood': 'tvl',
                    'ach_ggl': 'ton',
                    'ach_hq': 'zul',
                    'ach_bt': 'kal',
                    #'ach_aug_ood': 'pag',
                    'ach_aug_ggl': 'bak',#'cmn_Hant',
                    'ach_aug_hq': 'pus',
                    'ach_aug_bt': 'abk',
                    'lug_ood': 'pap',
                    'lug_ggl': 'hat',
                    'lug_hq': 'lug',#'mkd',
                    'lug_bt': 'tuk_Latn',
                    #'lug_aug_ood': 'yor',
                    'lug_aug_ggl': 'tuk',
                    'lug_aug_hq': 'sqi',
                    'lug_aug_bt': 'tir',                    
                    'luo_ood': 'mlg',
                    'luo_ggl': 'tur',
                    'luo_hq': 'ido_Latn',
                    'luo_bt': 'mai',
                    #'luo_aug_ood': 'ibo',
                    'luo_aug_ggl': 'heb',#'srp_Cyrl',
                    'luo_aug_hq': 'srp_Latn',
                    'luo_aug_bt': 'bos_Latn'#'kir_Cyrl',
                    
                    }
#['>>ewe<<','>>sna<<','>>lin<<','>>toi_Latn<<','>>ceb<<','>>oss<<','>>run<<','>>mfe<<','>>ilo<<','>>zlm_Latn<<','>>pes<<',
#'>>smo<<','>>hil<<','>>niu<<','>>sag<<','>>fij<<','>>cmn_Hans<<','>>nya<<','>>tso<<','>>war<<',
#'>>gil<<','>>hau_Latn<<','>>umb<<','>>glv<<','>>tvl<<','>>ton<<','>>zul<<','>>kal<<','>>pag<<','>>cmn_Hant<<',
#'>>pus<<','>>abk<<','>>pap<<','>>hat<<','>>mkd<<','>>tuk_Latn<<','>>yor<<','>>tuk<<','>>sqi<<','>>tir<<','>>mlg<<',
#'>>tur<<','>>ido_Latn<<','>>mai<<','>>ibo<<','>>srp_Cyrl<<','>>srp_Latn<<','>>kir_Cyrl<<','>>heb<<','>>bos_Latn<<',
#'>>bak<<','>>ast<<','>>som<<','>>tah<<','>>chv<<','>>kek_Latn<<','>>lug<<','>>vie<<',
#'>>wln<<','>>isl<<','>>hye<<','>>mah<<','>>yue_Hant<<','>>crh_Latn<<','>>amh<<','>>nds<<','>>pan_Guru<<','>>xho<<','>>ukr<<',
#'>>cat<<','>>afr<<','>>tat<<','>>guj<<','>>jpn<<','>>mon<<','>>eus<<','>>nob<<','>>glg<<','>>ind<<','>>sin<<','>>cym<<','>>zho_Hant<<',
#'>>zho_Hans<<','>>tgk_Cyrl<<','>>aze_Latn<<','>>ltz<<','>>bod<<','>>asm<<','>>tel<<','>>urd<<','>>kaz_Cyrl<<','>>lat_Latn<<','>>gla<<',
#'>>kan<<','>>bul<<','>>kin<<','>>ina_Latn<<','>>ron<<','>>spa<<','>>csb_Latn<<','>>iba<<','>>tha<<','>>nno<<','>>hrv<<','>>fry<<','>>bre<<',
#'>>mar<<','>>sme<<','>>swe<<','>>deu<<','>>jav<<','>>snd_Arab<<','>>ben<<','>>cmn<<','>>ces<<''>>ita<<','>>fin<<','>>por<<','>>hin<<',
#'>>hun<<','>>mal<<','>>pol<<','>>fra<<','>>nld<<','>>epo<<','>>slv<<','>>hsb<<','>>kur_Latn<<','>>ori<<','>>tam<<','>>bel<<','>>dan<<',
#'>>ara<<','>>mya<<','>>rus<<','>>mri<<','>>est<<','>>uzb_Latn<<','>>lao<<','>>yid<<','>>uzb_Cyrl<<','>>uig_Arab<<','>>lit<<','>>zho<<',
#'>>lav<<','>>ell<<','>>kat<<''>>gle<<','>>mlt<<','>>khm<<','>>oci<<','>>kur_Arab<<','>>ang_Latn<<','>>kaz_Latn<<','>>wol<<','>>sun<<',
#'>>chr<<','>>tat_Latn<<','>>mhr<<','>>tyv<<','>>rom<<','>>cha<<','>>kab<<','>>nav<<','>>arg<<','>>khm_Latn<<','>>bul_Latn<<','>>udm<<',
#'>>quc<<','>>cor<<','>>san_Deva<<','>>fao<<','>>bel_Latn<<','>>jbo_Latn<<','>>yue<<','>>grn<<','>>sco<<','>>arq<<','>>ltg<<','>>yue_Hans<<',
#'>>min<<''>>nan<<','>>bam_Latn<<','>>ido<<','>>ile_Latn<<','>>wuu<<','>>crh<<','>>tlh_Latn<<','>>lzh<<','>>jbo<<','>>lzh_Hans<<',
#'>>vol_Latn<<','>>lfn_Latn<<','>>arz<<']
    for r in replacements:
        if (f'>>{r}<<' not in tokenizer.encoder and
            f'>>{replacements[r]}<<' in tokenizer.encoder):
            tokenizer.encoder[f">>{r}<<"] = tokenizer.encoder[f">>{replacements[r]}<<"]
            del tokenizer.encoder[f">>{replacements[r]}<<"]

    # Check that all the evaluation language codes are mapped to something.
#     for r in config['eval_languages']:
#         if f'>>{r}<<' not in tokenizer.encoder:
#             raise ValueError(f'Language code {r} not found in the encoder.')

In [ ]:
train_data  = train_data_raw.map(
    preprocess,
    remove_columns=['source', 'target', 'source_language', 'target_language'],
    batched=True)

validation_data  = validation_data_raw.map(
    preprocess,
    remove_columns=['source', 'target', 'source_language', 'target_language'],
    batched=True)

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")
import os
os.environ["WANDB_API_KEY"] = secret_value_0
wandb.init(project=config['wandb_project'], config=config, entity="azawahry")

transformers.logging.set_verbosity_error()

trainer = transformers.Seq2SeqTrainer(
    model,
    config['train_settings'],
    train_dataset = train_data,
    eval_dataset = validation_data,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = lambda x: compute_metrics(
        x, config['eval_languages'][:-1], config['validation_samples_per_language']),
    callbacks = [
        transformers.EarlyStoppingCallback(
            early_stopping_patience = config['early_stopping_patience'])],
)

In [ ]:
# if config['eval_pretrained_model']:
#     trainer.evaluate()

In [ ]:
trainer.train()